In [1]:
#! pip install spacy

In [2]:
#! pip install tensorflow

In [3]:
#! python -m spacy download en_core_web_md -q

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import en_core_web_md
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
nlp = en_core_web_md.load()

2024-03-18 17:06:17.964404: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 17:06:18.990124: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Init

In [5]:
df = pd.read_csv('/workspaces/spam/spam.csv', encoding='ISO-8859-1')

In [6]:
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)

In [7]:
# Initialize the LabelEncoder
le = LabelEncoder()
# Fit label encoder and return encoded labels
encoded_labels = le.fit_transform(df['v1'])
df['encoding'] = encoded_labels

# Pre-processing

 * using the command `str.isalnum` remove all characters from your strings that are not alphanumeric except for whitespaces, and apostrophies.
 * using `str.replace`, `str.lower` and `str.strip` replace double whitespaces with single whitespaces, convert all characters to lowercase and trim starting and finishing whitespaces.
 * using spacy, replace all tokens in your texts with `lemma_` and remove all the stop words.

In [8]:
def clean_text(text):
    # Remove specified characters and symbols
    text = re.sub(r"[.,?!;:*&\-'/()]", "", text)  # Remove listed characters
    text = re.sub(r"\b[a-zA-Z]\b", "", text)  # Remove single letters
    text = re.sub(r"\s+", " ", text)  # Replace multiple spaces with a single space
    text = text.lower().strip()
    
    # Lemmatization and removing stopwords
    text = " ".join([token.lemma_ for token in nlp(text) if (token.lemma_ not in STOP_WORDS) and (token.text not in STOP_WORDS)])
    
    return text

In [9]:
df["v2_clean"] = df["v2"].apply(clean_text)

Are all the v2_clean the same type?

In [10]:
mask = df.v2_clean.apply(lambda x: type(x)==str)
mask.value_counts()

v2_clean
True    5572
Name: count, dtype: int64

let's check most_common words for more preprocessing and prep for tokenizer steps

In [11]:
words = df['v2_clean'].str.lower().str.split(expand=True).stack()

# Count words
word_count = Counter(words)

# Display the most n common words
print(word_count.most_common(20))

[('2', 480), ('m', 463), ('ur', 380), ('come', 322), ('4', 287), ('know', 283), ('s', 283), ('free', 278), ('ok', 277), ('lt#gt', 276), ('good', 273), ('send', 270), ('like', 252), ('want', 243), ('ill', 237), ('time', 235), ('day', 234), ('"', 232), ('love', 224), ('text', 214)]


In [12]:
specific_words = {'ur', 'come', 'know', 'm', 's', '2', '4', 'lt#gt', '"'}

# Function to remove specific words from a single text
def remove_specific_words(text):
    return " ".join(word for word in text.split() if word not in specific_words)

# Apply this function to each text in the 'v2_clean' column
df['v2_clean'] = df['v2_clean'].apply(remove_specific_words)


# Tokenize v2_clean

In [13]:
# Collect all words into a single list and then create a set to find unique words
unique_words = set(word for message in df.v2_clean for word in message.split())
# Calculate the total number of unique words
total_unique_words = len(unique_words)

In [14]:
total_words = df.v2_clean.apply(lambda x: len(x.split()))
total_words.sum(), total_unique_words

(41064, 8364)

In [15]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20000, oov_token="out_of_vocab")

In [16]:
#create the index for each words
tokenizer.fit_on_texts(df['v2_clean'])

I am trying to find the best num_words because if set too low some rare words will not be learned by the model

In [17]:
# Tokenizer's learned vocabulary
learned_words = set(tokenizer.word_index.keys())

# Dataset's vocabulary
dataset_words = set(word for text in df['v2_clean'] for word in text.lower().split())

# Find the unique OOV words by comparing the two sets
unique_oov_words = dataset_words - learned_words

print(f"Number of unique out_of_vocab words: {len(unique_oov_words)}")
print(f"Unique out_of_vocab words: {unique_oov_words}")

Number of unique out_of_vocab words: 249
Unique out_of_vocab words: {'\\not', '\\for', '\\nonenowhere', 'joy\\', 'doesdiscountshitinnit\\', '\\hello@drivby0quit', '\\jeevithathile', '\\how', '\\"she', 'life\\', 'late\\', '\\wen', '\\urgent', '\\gran', '\\smokes', '\\are', 'xxx\\', '\\gette', 'forever\\', '\\goodmorne', '\\shit', 'more\\', 'drive\\', 'wrk\\', 'credit\\', '@', 'adress\\', 'later\\', '\\nver', 'frndship\\', '\\sweet\\', '_', 'couch\\', 'xxxx\\', '\\hi', 'it+both', 'don\\', '\\"welcomes\\', 'monthlysubscription@50pmsg', '\\valentines', '@kiosk', 'shit\\', '\\the', '\\get', '\\kudi\\"yarasu', 'everything\\', '<', '\\pete', '\\im', '\\hurt', '\\oh', 'it\\', '\\ur', 'alone\\', '[', '=', '0a$network', '\\power', '\\checkmate\\', '\\hello\\', '\\"value', 'energy\\', 'crore\\', 'terry\\', 'msg@å£150rcvd', '\\speak', '\\it', 'ukp>2000', 'locaxx\\', 'l8r\\', '\\path', '@shesil', '\\what', 'mrng\\', '\\yeh', '\\crazy\\', 'love|', '+123', 'standing|', 'ì_ll', 'iwas+marinethatåõs', '

In [18]:
df["v2_encoded"] = tokenizer.texts_to_sequences(df['v2_clean'])

# Padding & split

In [19]:
df.head()

,v1,v2,encoding,v2_clean,v2_encoded
0,ham,"Go until jurong point, crazy.. Available only ...",0,jurong point crazy available bugis great world...,"[3606, 213, 424, 442, 921, 40, 187, 922, 2352,..."
1,ham,Ok lar... Joking wif u oni...,0,ok lar joke wif oni,"[3, 177, 443, 270, 1437]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry wkly comp win fa cup final tkts 21s...,"[2, 283, 510, 642, 22, 1438, 822, 402, 1439, 1..."
3,ham,U dun say so early hor... U c already then say...,0,dun early hor,"[109, 132, 2356]"
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,nah think usf live,"[683, 14, 643, 114]"


lets make sure all the v2_encoded have the same lenght

In [20]:
X = tf.keras.preprocessing.sequence.pad_sequences(df['v2_encoded'], padding="post")

In [21]:
y = df['encoding']

we have imbalanced -> stratify

In [22]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.3, stratify=y)

this will help use built in tensorflow method to shuffle and batch for training

In [23]:
train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val = tf.data.Dataset.from_tensor_slices((x_val, y_val))

In [24]:
train_batch = train.shuffle(len(train)).batch(64)
val_batch = val.shuffle(len(val)).batch(64)

In [25]:
 # Regardons un batch 
for X, y in train_batch.take(1):
  print(X, y)

tf.Tensor(
[[ 190   75 1409 ...    0    0    0]
 [ 896  111    0 ...    0    0    0]
 [  21    9   28 ...    0    0    0]
 ...
 [1060  828  548 ...    0    0    0]
 [1133  893    0 ...    0    0    0]
 [   2   60 6839 ...    0    0    0]], shape=(64, 59), dtype=int32) tf.Tensor(
[1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1], shape=(64,), dtype=int64)


2024-03-18 17:08:31.320459: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
